In [68]:
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import os
import json

In [4]:
response = rq.get("https://sammlungen.ub.uni-frankfurt.de/kolonialbibliothek/periodical/titleinfo/7720783")

In [5]:
response

<Response [200]>

In [6]:
response.text

'<!DOCTYPE html>\n<html lang="de"><head><title>Kolonialbibliothek / Deutsche Kolonialzeitung</title><meta name="twitter:image" content="http://sammlungen.ub.uni-frankfurt.de/kolonialbibliothek/download/webcache/304/7720797"><meta name="twitter:title" content="Deutsche Kolonialzeitung&#160;: Organ der Deutschen Kolonialgesellschaft. M&#252;nchen : Deutsche Kolonialgesellschaft, 1.1884 - 39.1922, 1884-1943"><meta property="og:site" content="Kolonialbibliothek"><meta property="og:url" content="http://sammlungen.ub.uni-frankfurt.de/kolonialbibliothek/7720783"><meta property="og:type" content="book"><meta property="og:title" content="Deutsche Kolonialzeitung : Organ der Deutschen Kolonialgesellschaft"><meta property="book:release_date" content="1922"><meta property="og:image" content="http://sammlungen.ub.uni-frankfurt.de/kolonialbibliothek/download/webcache/304/7720797"><meta property="og:image:width" content="304"><meta property="og:image:height" content="430"><meta http-equiv="Content-Ty

In [23]:
df_jahrgänge = pd.read_html(response.content, attrs = {"id": "titleInfoJournal_volumes"}, extract_links= "all")[0]

In [35]:
df_jahrgänge[0].values[3]

('PDF\xa02 (1885)', '/download/pdf/7735720.pdf')

In [39]:
soup = BeautifulSoup(response.text, "html.parser")
table = soup.find("table", id = "titleInfoJournal_volumes")
table

<table cellspacing="0" class="titleInfo titleAdd columns titleInfoMetadataChildren" id="titleInfoJournal_volumes"><tr><td class="tdSubheader" colspan="3"><div class="subheader" role="heading">Jahrgänge</div></td></tr><tr><tr><td><div class="titleLinks -title"><a class="childLink" href="/kolonialbibliothek/periodical/titleinfo/7720784"><div class="title journal_volume"><span class="partInfo">1 (1884)</span></div></a></div></td></tr><tr><td><div class="titleLinks -title"><a class="resourceLink classic inlineLink" href="/download/pdf/7735720.pdf" title=" [277,67 mb]"><span>PDF<span class="hide-underlined"> </span><span class="ln-ico ln-pdf"></span></span></a><a class="childLink" href="/kolonialbibliothek/periodical/titleinfo/7735720"><div class="title journal_volume"><span class="partInfo">2 (1885)</span></div></a></div></td></tr><tr><td><div class="titleLinks -title"><a class="resourceLink classic inlineLink" href="/download/pdf/7735721.pdf" title=" [303,41 mb]"><span>PDF<span class="hid

In [53]:
links = []
for row in table.find_all("tr")[1:]:  # skip header row
    link_tag = row.find("a", class_="childLink")
    if link_tag and link_tag.has_attr("href"):
        links.append(link_tag["href"])
    else:
        links.append(None)

list_ids = [link.split("/")[-1] for link in links]
unique_ids = sorted(list(set(list_ids)))

In [56]:
"https://sammlungen.ub.uni-frankfurt.de/i3f/v20/" + unique_ids[0] + "/manifest"

'https://sammlungen.ub.uni-frankfurt.de/i3f/v20/7720784/manifest'

In [63]:
manifest_urls = ["https://sammlungen.ub.uni-frankfurt.de/i3f/v20/" + id + "/manifest" for id in unique_ids][0:1]
manifest_urls

['https://sammlungen.ub.uni-frankfurt.de/i3f/v20/7720784/manifest']

In [65]:
for link in manifest_urls:
    response_iiif = rq.get(link)
    data = response_iiif.json()


In [74]:
data["label"]

'Deutsche Kolonialzeitung: 1 (1884)'

In [76]:
filename = os.path.join("output", (data["label"] + ".json").replace(" ", "_").replace(":", ""))
with open(filename, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

filename

'output\\Deutsche_Kolonialzeitung_1_(1884).json'